## 진행상황

MH_11/08
1. FLAG_MOBIL : drop
2. DAYS_BIRTH, DAYS_EMPLOYED abs.() 절댓값 처리
3. occyp_type fillna(Nan)적용
4. ID 생성 후 ordinalencoder 적용
5. family_size 대치 & child Y or N로 적용
6. ability 사용
7. MH_catboost 사용(cat_cols)
8. income_total >> log_scale
9. 나머지 : standard_scale, 

1st 결과
- public 0.98, private 0.96

## 필요한 라이브러리 호출

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
from sklearn.preprocessing import OrdinalEncoder


warnings.filterwarnings('ignore')

## 데이터 전처리 과정

In [2]:
# 경로 설정 (dataset을 저장하는 경로임.)
path = 'C:/Users/smh81/Documents/Dacon_2/Dataset/'

train = pd.read_csv('{}train.csv'.format(path))
test = pd.read_csv('{}test.csv'.format(path))

In [3]:
#쓸모없는 칼럼 삭제
for df in (train,test):
    df.drop(['FLAG_MOBIL'], axis=1, inplace=True)   
    df.drop(columns=['index'], inplace=True)
    
del df

### 1. 나이, 고용일자 절댓값 처리

In [4]:
# 1st

def year_month_total(df_1, df_2, column):
 
    # 변수 설정  
    df_list = []
    df_list.append(df_1)
    df_list.append(df_2)
    names = ['train', 'test']
    
    # 입력한 컬럼이 DAYS_BIRTH인 경우 If로 처리
    
    if (column == 'DAYS_BIRTH') == True:    
        for dataset in df_list:
            dataset['{}'.format(column)] = dataset['{}'.format(column)].abs()
    
    # 입력한 컬럼이 DAYS_EMPLOYED인 경우 else로 처리 (occyp_type 처리를 위한 작업이므로 1번째로 수행)
    
    else:
        for dataset in df_list:
            # plus_num = dataset[dataset.loc[:,column]>0].DAYS_EMPLOYED
            # for i in plus_num.unique():
            #     dataset[column] = dataset.loc[:,column].replace(i, 0)

            dataset[column] = dataset[column].replace(365243, 0)
            
            dataset['{}'.format(column)] = dataset['{}'.format(column)].abs()
           
    del df_list,names
    return dataset.head(3)

    

In [5]:
year_month_total(train, test,'DAYS_BIRTH')

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month
0,M,Y,N,0,112500.0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,21990,365243,0,1,0,NaN,2.0,-60.0
1,F,N,Y,0,135000.0,State servant,Higher education,Married,House / apartment,18964,-8671,0,1,0,Core staff,2.0,-36.0
2,F,N,Y,0,69372.0,Working,Secondary / secondary special,Married,House / apartment,15887,-217,1,1,0,Laborers,2.0,-40.0


In [6]:
year_month_total(train, test,'DAYS_EMPLOYED')

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month
0,M,Y,N,0,112500.0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,21990,0,0,1,0,NaN,2.0,-60.0
1,F,N,Y,0,135000.0,State servant,Higher education,Married,House / apartment,18964,8671,0,1,0,Core staff,2.0,-36.0
2,F,N,Y,0,69372.0,Working,Secondary / secondary special,Married,House / apartment,15887,217,1,1,0,Laborers,2.0,-40.0


### 2. 직업 Nan 값 처리

In [7]:
# 2nd

def occyp_type_nan(df_1, df_2):
    """[직업 컬럼 내의 Nan 값을 처리합니다.]

    Args:
        df_1 ([DataFrame]): [데이터 프레임 이름을 입력해주세요]
        df_2 ([DataFrame]): [데이터 프레임 이름을 입력해주세요]
    """
    # 변수 설정
    
    df_list = []
    df_list.append(df_1)
    df_list.append(df_2)
    names = ['train', 'test']

    # 무직자 처리 함수
    
    for dataset,name in zip(df_list,names):
        dataset['occyp_type'] = dataset['occyp_type'].fillna('Nan')
        
        # print('직업 컬럼의 ' +
        #       '\033[92m' + 'Nan value {}명 중 '.format(len(dataset.loc[dataset['occyp_type'] == 'Nan'])) +
        #       '\033[96m' + '{}명이 실제 고용일수가 0일입니다. '.format(len(dataset.loc[(dataset['occyp_type'] == 'Nan') & (dataset['DAYS_EMPLOYED'] == 0)])) + 
        #       '\033[0m'+ '따라서 jobless로 처리합니다.')
        
        # dataset.loc[(dataset['occyp_type'] == 'Nan') & (dataset['DAYS_EMPLOYED'] == 0), 'occyp_type'] = 'jobless'
        
        # # 무직자는 아니지만 데이터가 없는 경우
        
        # print('나머지 {}명 데이터는 no_data로 처리합니다.'.format(len(dataset.loc[dataset['occyp_type'] == 'Nan'])))
        
        # dataset.loc[dataset['occyp_type'] == 'Nan', 'occyp_type'] = 'no data'
        
        # print('{}set의 직업 Nan value를 처리하였습니다.\n'.format(name))
        
    del df_list, names
    return dataset.head(5)

occyp_type_nan(train, test)

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month
0,M,Y,N,0,112500.0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,21990,0,0,1,0,Nan,2.0,-60.0
1,F,N,Y,0,135000.0,State servant,Higher education,Married,House / apartment,18964,8671,0,1,0,Core staff,2.0,-36.0
2,F,N,Y,0,69372.0,Working,Secondary / secondary special,Married,House / apartment,15887,217,1,1,0,Laborers,2.0,-40.0
3,M,Y,N,0,112500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,19270,2531,1,0,0,Drivers,2.0,-41.0
4,F,Y,Y,0,225000.0,State servant,Higher education,Married,House / apartment,17822,9385,1,0,0,Managers,2.0,-8.0


### 3. ID컬럼 생성

In [8]:
def ID_col(df_1, df_2):
    """[데이터 셋 전처리 함수입니다.]

    Args:
        df_1 ([Dataset]): [train 데이터셋을 입력해주세요]
        df_2 ([Dataset]): [test 데이터셋을 입력해주세요]
    """
    
    # 변수 설정
     
    df_list = []
    df_list.append(df_1)
    df_list.append(df_2)
    a = []
    change_credit = 0
    # names = ['train', 'test']
    
    # ID 생성
    
    # for name, dataset, in zip(names, df_list):
    for dataset in df_list:
        dataset['ID'] = dataset['gender'].astype(str) +'.'+ dataset['car'].astype(str)+'.' + dataset['reality'].astype(str)+'.' + \
                dataset['child_num'].astype(str)+'.' + dataset['income_total'].astype(str) +'.'+ dataset['income_type'].astype(str) +'.'+ \
                dataset['edu_type'].astype(str)+'.' + dataset['family_type'].astype(str)+'.' + dataset['house_type'].astype(str) +'.'+ \
                dataset['DAYS_BIRTH'].astype(str) +'.'+ dataset['DAYS_EMPLOYED'].astype(str) +'.'+ dataset['work_phone'].astype(str) +'.'+ \
                dataset['phone'].astype(str) +'.'+ dataset['email'].astype(str) +'.' + dataset['family_size'].astype(str)+'.' + dataset['occyp_type'].astype(str)
        
        dataset = dataset.sort_values(['ID','begin_month'], ascending=[True, False])
        dataset.reset_index(inplace=True)
        dataset.drop(columns={'index'}, inplace=True)
        dataset['begin_month'] = dataset['begin_month'].abs()


        # OrdinalEncoder로 ID 변환
        
        print('\033[96m' + '해당 Data 내의 고유 ID 수는 {}입니다. '.format(dataset['ID'].nunique()) + 
            '\033[0m' + 'OrdinalEncoder를 이용힌 변환을 시작합니다.')
        encode = OrdinalEncoder()
        dataset[['ID']] = encode.fit_transform(dataset[['ID']])

        # Nth_card 컬럼 추가 (ID가 같은데 카드 개수가 여러 개인 사람을 대상으로 몇 번째 카드를 만들었는지 생성해준다)
        
        # print('Column(Nth_card)을 생성합니다.')
        # for i in tqdm(range(dataset['ID'].nunique())):
        #     Nth_card = len(dataset[dataset['ID']==i]) + 1
            
        #     while Nth_card > 0:
        #         Nth_card = Nth_card -1
        #         a.append(Nth_card)
        #         if Nth_card == 1:
        #             break
                
        # dataset['Nth_card'] = a        
        # a.clear()
        # print('\033[92m' + '신규로 카드를 개설한 사람의 수 : {}'.format(dataset[dataset['begin_month']==0.0]['ID'].nunique()) +
        #     '\033[0m')
        
        # Credit이 좋아지지 않고 악화된 경우를 체크하는 부분 (begin_month기준으로 Nth_card를 추가해줬으므로 인덱스 값을 기준으로 시행한다.)
        
        if (dataset['ID'].nunique() > 8000)==True:
            print('credit 정보를 확인하고 시간이 지나며 credit이 악화된 사례가 있는지 확인합니다.')
            for i in tqdm(range(dataset['ID'].nunique())):
                num = dataset.loc[dataset['ID'] == i]['credit'].to_list()
                for j in range(len(num)-1):
                    if (num[j] <= num[j+1]) == True:
                        pass
                    else :
                        change_credit += 1
                        break
            print('\033[96m'+'{}명 (전체 인원 중 {:.2f}%)은 credit이 개선되지 않고 악화된 사례가 존재합니다.'.format(change_credit, 
                                                                        (change_credit/dataset['ID'].nunique())*100) + 
                '\033[0m')
            # dataset 저장
            print('dataset(user_list_train)을 저장합니다.')    
            dataset.to_csv('{}user_list_train.csv'.format(path), index=False)
        else:
            print('\033[96m' + 'credit 정보가 없는 testset입니다.' + 
                '\033[0m')
            # dataset 저장
            print('dataset(user_list_test)을 저장합니다.')    
            dataset.to_csv('{}user_list_test.csv'.format(path), index=False)
        
    # del names, a
    del a
    return dataset.head(5)

In [9]:
ID_col(train, test)

  0%|          | 0/8759 [00:00<?, ?it/s]

해당 Data 내의 고유 ID 수는 8759입니다. OrdinalEncoder를 이용힌 변환을 시작합니다.
credit 정보를 확인하고 시간이 지나며 credit이 악화된 사례가 있는지 확인합니다.


100%|██████████| 8759/8759 [00:08<00:00, 988.41it/s] 


1842명 (전체 인원 중 21.03%)은 credit이 개선되지 않고 악화된 사례가 존재합니다.
dataset(user_list_train)을 저장합니다.
해당 Data 내의 고유 ID 수는 5585입니다. OrdinalEncoder를 이용힌 변환을 시작합니다.
credit 정보가 없는 testset입니다.
dataset(user_list_test)을 저장합니다.


,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month,ID
0,F,N,N,0,103500.0,Pensioner,Higher education,Married,House / apartment,21701,0,0,1,0,Nan,2.0,3.0,0.0
1,F,N,N,0,103500.0,Pensioner,Higher education,Married,House / apartment,21701,0,0,1,0,Nan,2.0,46.0,0.0
2,F,N,N,0,103500.0,Pensioner,Higher education,Married,House / apartment,21701,0,0,1,0,Nan,2.0,53.0,0.0
3,F,N,N,0,103500.0,Pensioner,Higher education,Single / not married,House / apartment,24543,0,0,0,0,Nan,1.0,28.0,1.0
4,F,N,N,0,103500.0,Pensioner,Secondary / secondary special,Married,House / apartment,20123,0,0,0,0,Nan,2.0,11.0,2.0


In [10]:
train = pd.read_csv('{}user_list_train.csv'.format(path))
test = pd.read_csv('{}user_list_test.csv'.format(path))

print(train.shape)
print(test.shape)

(26457, 19)
(10000, 18)


### 4.faimly_size, child_num 처리

In [11]:
def replace_value_family(df_1, df_2, num_of_outlier):
    """[Family_size의 outlier를 대치해주는 작업을 시행합니다.]

    Args:
        df_1 ([DataFrame]): [데이터 프레임 이름을 입력해주세요]
        df_2 ([DataFrame]): [데이터 프레임 이름을 입력해주세요]
        num_of_outlier ([outlier_standard]): [family_size의 outlier 기준을 입력하세요]
    """
    # 변수 설정
    
    df_list = []
    df_list.append(df_1)
    df_list.append(df_2)
    names = ['train', 'test']

    # family_size Outlier를 처리하는 작업
    
    for dataset,name in zip(df_list,names):
        
        print('child_num 컬럼 Y/N 대치.')
        dataset.loc[dataset['child_num'] ==0, 'child_num'] = 'N'
        dataset.loc[dataset['child_num'] != 'N', 'child_num'] = 'Y'
        
        
        print('\033[96m' + 'family_size {}명 이상인 사람의 수 : {}'.format(num_of_outlier, 
                                                                      len(dataset.loc[dataset['family_size'] >= num_of_outlier])) +
              '\033[0m')
        
        sub = dataset.loc[dataset['family_size']>= num_of_outlier]
        
        for i in tqdm(range(len(sub))):
        
            a = int(dataset[dataset['family_type'] == sub['family_type'].values[i]].mean()['family_size'])
            
            dataset.loc[(dataset['family_size']>= num_of_outlier), 'family_size'] = a
        
        # 잘 처리되었는지 확인하는 부분
        
        print('\033[96m' + 'family_size {}명 이상인 사람의 수 : {}'.format(num_of_outlier, 
                                                        len(dataset.loc[dataset['family_size'] >= num_of_outlier]))+
              '\033[0m')

        # family_size를 조정하고 난 후에 1인당 소득으로 total_income을 scale_down 해줌.
        
        print('1인당 소득으로 소득 수준을 조정합니다.')
        
        # dataset['income_mean'] = dataset['income_total'] / dataset['family_size']
        
        print('{} 데이터 셋 처리 완료.\n'.format(name))
    
    del sub,df_list, names
    return dataset.head(1)

replace_value_family(train, test, 7)

  0%|          | 0/15 [00:00<?, ?it/s]

child_num 컬럼 Y/N 대치.
family_size 7명 이상인 사람의 수 : 15


 20%|██        | 2/10 [00:00<00:00, 11.22it/s]

family_size 7명 이상인 사람의 수 : 0
1인당 소득으로 소득 수준을 조정합니다.
train 데이터 셋 처리 완료.

child_num 컬럼 Y/N 대치.
family_size 7명 이상인 사람의 수 : 10


100%|██████████| 10/10 [00:00<00:00, 11.21it/s]


family_size 7명 이상인 사람의 수 : 0
1인당 소득으로 소득 수준을 조정합니다.
test 데이터 셋 처리 완료.



,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month,ID
0,F,N,N,N,103500.0,Pensioner,Higher education,Married,House / apartment,21701,0,0,1,0,Nan,2.0,3.0,0.0


### 5. ability 파생변수 추가 

In [12]:
# #ability: 소득/(살아온 일수+ 근무일수)
for dataset in [train,test]:
    dataset['ability'] = dataset['income_total'] / (dataset['DAYS_BIRTH'] + dataset['DAYS_EMPLOYED'])
    
train.head(5)

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month,credit,ID,ability
0,F,N,N,N,103500.0,Pensioner,Higher education,Married,House / apartment,21701,0,0,1,0,Nan,2.0,24.0,0.0,0.0,4.769365
1,F,N,N,N,103500.0,Pensioner,Higher education,Married,House / apartment,21701,0,0,1,0,Nan,2.0,28.0,0.0,0.0,4.769365
2,F,N,N,N,103500.0,Pensioner,Higher education,Married,House / apartment,21701,0,0,1,0,Nan,2.0,30.0,2.0,0.0,4.769365
3,F,N,N,N,103500.0,Pensioner,Higher education,Single / not married,House / apartment,24543,0,0,0,0,Nan,1.0,17.0,2.0,1.0,4.217088
4,F,N,N,N,103500.0,Pensioner,Higher education,Single / not married,House / apartment,24543,0,0,0,0,Nan,1.0,39.0,2.0,1.0,4.217088


final_set.csv 저장
-----

In [13]:
names = ['train', 'test']

for name,dataset in zip(names, [train, test]):
    # dataset.drop(columns=['DAYS_BIRTH','DAYS_EMPLOYED','DAYS_BIRTH_MOD','DAYS_EMPLOYED_MOD'], inplace=True)
    dataset.to_csv('{}final_set_{}.csv'.format(path,name), index=False)